# Import Datasets

## WONDER data

In [1]:
import pandas as pd

# Load the text file as a DataFrame with appropriate separator (assuming tab-separated for .txt)
wonder = pd.read_csv('Provisional Mortality Statistics, 2018 through Last Week.txt', sep='\t')

# Drop extraneous columns
wonder = wonder.drop(columns=['Notes', 'Residence State Code', 'Population', 'Crude Rate', 'Age Adjusted Rate'])

# Convert 'Month Code' to a datetime object with 'M' monthly period for standardization
wonder['Month Code'] = pd.to_datetime(wonder['Month Code'], format='%Y/%m')

# Rename 'Residence State' column to 'State Name'
wonder = wonder.rename(columns={'Residence State': 'State Name'})

# Rename 'Month Code' column to 'Date'
wonder = wonder.rename(columns={'Month Code': 'Date'})

# Find latest date
latest_date = wonder['Date'].max()

# Print number of rows in DataFrame
print(f"Number of rows: {len(wonder):,}")

# Print latest date as month and year only
print(f"Latest record date: {latest_date.strftime('%B %Y')}")

# Rename 'month' column to 'MonthNote'
wonder = wonder.rename(columns={'Month': 'MonthNote'})

# Create provisional flag if 'Provisional' in 'MonthNote'
wonder['Provisional'] = wonder['MonthNote'].str.contains('Provisional', case=False)

# Create partial flag if 'Partial' in 'MonthNote'
wonder['Partial'] = wonder['MonthNote'].str.contains('Partial', case=False)

# Drop 'MonthNote' column
wonder = wonder.drop(columns=['MonthNote'])

# Filter out 2024 records, which are incomplete/partial
wonder = wonder[wonder['Date'] <= '2023-12-31']

wonder.head()

Number of rows: 3,870
Latest record date: July 2024


,Date,State Name,Deaths,Provisional,Partial
0,2018-01-01,Alabama,56.0,False,False
1,2018-01-01,Alaska,11.0,False,False
2,2018-01-01,Arizona,142.0,False,False
3,2018-01-01,Arkansas,36.0,False,False
4,2018-01-01,California,479.0,False,False


In [2]:
wonder

,Date,State Name,Deaths,Provisional,Partial
0,2018-01-01,Alabama,56.0,False,False
1,2018-01-01,Alaska,11.0,False,False
2,2018-01-01,Arizona,142.0,False,False
3,2018-01-01,Arkansas,36.0,False,False
4,2018-01-01,California,479.0,False,False
...,...,...,...,...,...
3485,2023-12-01,Vermont,23.0,True,False
3486,2023-12-01,Virginia,175.0,True,False
3487,2023-12-01,Washington,294.0,True,False
3488,2023-12-01,West Virginia,125.0,True,False


In [3]:
# Create a DataFrame to store peak overdose data
peakod = pd.DataFrame()

# Filter the dataset for records from January 1, 2020, onwards
filtered_wonder = wonder[wonder['Date'] >= '2020-01-01']

# Group data by 'State Name'
groups = filtered_wonder.groupby("State Name")

# Iterate through each group for individual state-level processing
for state, group in groups:
    # Find the maximum deaths for the group
    max_deaths = group['Deaths'].max()
    
    # Filter rows where deaths match the maximum
    max_rows = group[group['Deaths'] == max_deaths]
    
    # Select the most recent date among tied rows
    recent_row = max_rows[max_rows['Date'] == max_rows['Date'].max()]
    
    # Append the resulting row to peakod
    peakod = pd.concat([peakod, recent_row[['Deaths', 'Date', 'State Name']]])

# Reset index for clean DataFrame
peakod.reset_index(drop=True, inplace=True)

# Rename 'Deaths' column to 'Peakdeaths'
peakod.rename(columns={'Deaths': 'Peakdeaths'}, inplace=True)

# Save the resulting peakod DataFrame to a CSV file
peakod.to_csv('peakod.csv', index=False)

# Output the resulting DataFrame
peakod

,Peakdeaths,Date,State Name
0,159.0,2023-05-01,Alabama
1,51.0,2023-12-01,Alaska
2,372.0,2023-07-01,Arizona
3,70.0,2022-03-01,Arkansas
4,1105.0,2023-07-01,California
5,181.0,2021-08-01,Colorado
6,149.0,2021-07-01,Connecticut
7,62.0,2022-05-01,Delaware
8,49.0,2020-05-01,District of Columbia
9,760.0,2020-05-01,Florida


<hr>

## Predicted NVSS data 

In [22]:
import pickle as pickle

df = pd.read_csv("data_2025-01-15_16-23-29.csv", low_memory=False)

# Combine Year and Month into a 'Date' column and convert it to a datetime object
df['Date'] = pd.to_datetime(df['Year'].astype(str) + ' ' + df['Month'], format='%Y %B')

# Find latest date
latest_date = df['Date'].max()

# Print number of rows in DataFrame
print(f"Number of rows: {len(df):,}")

# Print latest date as month and year only
print(f"Latest record date: {latest_date.strftime('%B %Y')}")

# Save the latest date to a pickle file
with open('latest_date.pkl', 'wb') as f:
    pickle.dump(latest_date, f)

Number of rows: 69,600
Latest record date: August 2024


In [5]:
# Filter the DataFrame to include only rows where 'State' is 'US'
nvss = df.loc[df['State'] != 'US']

# Drop columns not needed for analysis
nvss = nvss.drop(columns=['Period', 'Footnote', 'Footnote Symbol'])

# Filter to include only Number of Drug Overdose Deaths
nvss = nvss.loc[nvss['Indicator'] == 'Number of Drug Overdose Deaths']

# Convert 'Date' column back to a datetime object
nvss['Date'] = pd.to_datetime(nvss['Date'])

# Remove comma from 'Predicted Value' column
nvss['Predicted Value'] = nvss['Predicted Value'].str.replace(',', '')

# Convert 'Predicted Value' column to float
nvss['Predicted Value'] = nvss['Predicted Value'].astype(float)

# Add 'New York City' into 'New York' values for 'Data Value'
nvss.loc[nvss['Data Value'] == 'New York City', 'Data Value'] = 'New York'

# Drop rows where 'Data Value' is 'New York City'
nvss = nvss[nvss['State Name'] != 'New York City']

# Drop 'Indicator' column
nvss = nvss.drop(columns=['Indicator'])

# Display the updated DataFrame
nvss.head()

,State,Year,Month,Data Value,Percent Complete,Percent Pending Investigation,State Name,Predicted Value,Date
10,AK,2015,April,126,100.0,0.0,Alaska,129.0,2015-04-01
15,AK,2015,August,124,100.0,0.0,Alaska,127.0,2015-08-01
26,AK,2015,December,121,100.0,0.0,Alaska,124.0,2015-12-01
45,AK,2015,February,127,100.0,0.0,Alaska,130.0,2015-02-01
58,AK,2015,January,126,100.0,0.0,Alaska,129.0,2015-01-01


<hr>

## State population data

Using intercensal estimated populations for rate calculations.

Annual Estimates of the Resident Population for the United States, Regions, States, District of Columbia, and Puerto Rico: April 1, 2020 to July 1, 2024 (NST-EST2024-POP)			
Source: U.S. Census Bureau, Population Division						
Release Date: December 2024						

In [6]:
# Read the unpacked population data into a DataFrame
population_df = pd.read_excel('NST-EST2024-POP.xlsx', header=3)

# Rename column 0 to 'State'
population_df = population_df.rename(columns={population_df.columns[0]: 'State Name'})

# Trim leading period from column 1
population_df['State Name'] = population_df['State Name'].str.lstrip('.')

# Drop column 2
population_df = population_df.drop(columns=[population_df.columns[1]])

# Drop if 'State Name' is 'United States', or 'Puerto Rico' or "Northeast" or "Midwest" or "South" or "West"
population_df = population_df[~population_df['State Name'].isin(['United States', 'Puerto Rico', 'Northeast', 'Midwest', 'South', 'West'])]

# Append 'y' before column name to all columns except 'State Name'
population_df = population_df.add_prefix('y')
population_df = population_df.rename(columns={'yState Name': 'State Name'})

# drop rows with missing values
population_df = population_df.dropna()

# Display the first few rows of the population data
population_df.head()

,State Name,y2020,y2021,y2022,y2023,y2024
5,Alabama,5033094.0,5049196.0,5076181.0,5117673.0,5157699.0
6,Alaska,733017.0,734420.0,734442.0,736510.0,740133.0
7,Arizona,7187135.0,7274078.0,7377566.0,7473027.0,7582384.0
8,Arkansas,3014546.0,3026870.0,3047704.0,3069463.0,3088354.0
9,California,39521958.0,39142565.0,39142414.0,39198693.0,39431263.0


In [7]:
# Pivot `population_df` from wide to long format
population_long = population_df.melt(id_vars=['State Name'], var_name='Year', value_name='Population')

# Drop 'y' in 'Year' column
population_long['Year'] = population_long['Year'].str.lstrip('y')

# Generate rows for each month for each Year-State dyad
population_expanded = population_long.loc[population_long.index.repeat(12)].reset_index(drop=True)
population_expanded['Month'] = population_expanded.groupby(['State Name', 'Year']).cumcount() + 1

# Correctly format the date column using .apply on Series level for zfill on Month
population_expanded['Date'] = population_expanded.apply(
    lambda row: pd.to_datetime(f"{row['Year']}-{str(row['Month']).zfill(2)}-01"), axis=1
)

# Sort the data by State Name and date
population_expanded = population_expanded.sort_values(by=['State Name', 'Date']).reset_index(drop=True)

# Drop 'Month' and 'Year' columns
population_expanded = population_expanded.drop(columns=['Month', 'Year'])

# Use 'latest_date' to set max 'Date' in population_expanded
population_expanded = population_expanded[population_expanded['Date'] <= latest_date]

# Display the updated population_expanded DataFrame
population_expanded.head()

,State Name,Population,Date
0,Alabama,5033094.0,2020-01-01
1,Alabama,5033094.0,2020-02-01
2,Alabama,5033094.0,2020-03-01
3,Alabama,5033094.0,2020-04-01
4,Alabama,5033094.0,2020-05-01


<hr>

# Merge Datasets

## Mini Codebook

NVSS origin

Data Value  is 'actual' 12 month cumulative sum of all drug overdoses

Percent Complete and Percent Pending are from NVSS as of date of download

➡️ Predicted Value is the lag-adjusted 12 month cumulative sum of all drug overdoses from NVSS

WONDER origin

➡️ Deaths is WONDER deaths for given month with underlying COD of drug poisoning, all intents

Provisional and Partial are flags for completeness of final reporting as of date of download

Census origin (not included)

Population is intercensal estimate for July 1st of each state-year

## Date restriction

Limit to January 2018 going forward to align with earliest WONDER data

In [8]:
# Filter the WONDER dataset to include data only from January 2018 onward
wonder = wonder[wonder['Date'] >= '2018-01-01']

# Filter the NVSS dataset to include data only from January 2018 onward
nvss = nvss[nvss['Date'] >= '2018-01-01']

# Display the filtered datasets
print(f"Filtered WONDER dataset: {wonder.shape}")
print(f"Filtered NVSS dataset: {nvss.shape}")

Filtered WONDER dataset: (3490, 5)
Filtered NVSS dataset: (4080, 9)


## Merge

In [9]:
merged_df = wonder.merge(nvss, on=['State Name', 'Date'], how='outer')
# 3-way merge
# merged_df = population_expanded.merge(nvss, on=['State Name', 'Date'], how='outer').merge(wonder, on=['State Name', 'Date'], how='outer')

merged_df

,Date,State Name,Deaths,Provisional,Partial,State,Year,Month,Data Value,Percent Complete,Percent Pending Investigation,Predicted Value
0,2018-01-01,Alabama,56.0,False,False,AL,2018,January,785,100.0,0.150636,808.0
1,2018-01-01,Alaska,11.0,False,False,AK,2018,January,138,100.0,0.000000,141.0
2,2018-01-01,Arizona,142.0,False,False,AZ,2018,January,"1,571",100.0,0.245040,1624.0
3,2018-01-01,Arkansas,36.0,False,False,AR,2018,January,427,100.0,0.052645,437.0
4,2018-01-01,California,479.0,False,False,CA,2018,January,"5,092",100.0,0.320261,5285.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4075,2024-01-01,Wyoming,NaN,NaN,NaN,WY,2024,January,126,100.0,0.000000,132.0
4076,2024-07-01,Wyoming,NaN,NaN,NaN,WY,2024,July,125,100.0,0.019260,127.0
4077,2024-06-01,Wyoming,NaN,NaN,NaN,WY,2024,June,127,100.0,0.019212,130.0
4078,2024-03-01,Wyoming,NaN,NaN,NaN,WY,2024,March,129,100.0,0.000000,133.0


In [10]:
# Drop rows where all values are NaN or NaT
merged_df = merged_df.dropna(how='all')

# Display the updated DataFrame
merged_df

,Date,State Name,Deaths,Provisional,Partial,State,Year,Month,Data Value,Percent Complete,Percent Pending Investigation,Predicted Value
0,2018-01-01,Alabama,56.0,False,False,AL,2018,January,785,100.0,0.150636,808.0
1,2018-01-01,Alaska,11.0,False,False,AK,2018,January,138,100.0,0.000000,141.0
2,2018-01-01,Arizona,142.0,False,False,AZ,2018,January,"1,571",100.0,0.245040,1624.0
3,2018-01-01,Arkansas,36.0,False,False,AR,2018,January,427,100.0,0.052645,437.0
4,2018-01-01,California,479.0,False,False,CA,2018,January,"5,092",100.0,0.320261,5285.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4075,2024-01-01,Wyoming,NaN,NaN,NaN,WY,2024,January,126,100.0,0.000000,132.0
4076,2024-07-01,Wyoming,NaN,NaN,NaN,WY,2024,July,125,100.0,0.019260,127.0
4077,2024-06-01,Wyoming,NaN,NaN,NaN,WY,2024,June,127,100.0,0.019212,130.0
4078,2024-03-01,Wyoming,NaN,NaN,NaN,WY,2024,March,129,100.0,0.000000,133.0


## Data quality checks

In [11]:
rows_per_state = merged_df['State Name'].value_counts()
rows_per_state

State Name
Alabama                 80
Alaska                  80
Arizona                 80
Arkansas                80
California              80
Colorado                80
Connecticut             80
Delaware                80
District of Columbia    80
Florida                 80
Georgia                 80
Hawaii                  80
Idaho                   80
Illinois                80
Indiana                 80
Iowa                    80
Kansas                  80
Kentucky                80
Louisiana               80
Maine                   80
Maryland                80
Massachusetts           80
Michigan                80
Minnesota               80
Mississippi             80
Missouri                80
Montana                 80
Nevada                  80
New Hampshire           80
New Jersey              80
New Mexico              80
New York                80
North Carolina          80
North Dakota            80
Ohio                    80
Oklahoma                80
Oregon           

In [12]:
# Extract the state names as a list and sort alphabetically
state_names = sorted(merged_df['State Name'].unique().tolist())

### Duplicates check

In [13]:
duplicates_check = merged_df.duplicated(subset=['State Name', 'Date']).any()
duplicates_check

False

<hr>

# Reconstruct 12-month cumulative

In [14]:
# Create a copy of merged_df for the calculation
filtered_df = merged_df.copy()

# Calculate the rolling sum of 'Deaths' with a window of 12 months for each state
filtered_df['recreated12m'] = filtered_df.groupby('State Name')['Deaths'].rolling(window=12, min_periods=1).sum().reset_index(0, drop=True)

# Now filter the filtered_df to limit the data to the time period Jan 2019 to Dec 2022 when the WONDER data was available to cumsum
filtered_df = filtered_df[(filtered_df['Date'] >= '2019-01-01') & (filtered_df['Date'] <= '2022-12-01')]

# Extract only the relevant columns and merge the calculated 'recreated12m' back into the original DataFrame
merged_df = merged_df.merge(filtered_df[['State Name', 'Date', 'recreated12m']], on=['State Name', 'Date'], how='left')

# Sort the copied DataFrame by State Name and Date in ascending order
merged_df = merged_df.sort_values(by=['State Name', 'Date'])

# Display the updated DataFrame
merged_df

,Date,State Name,Deaths,Provisional,Partial,State,Year,Month,Data Value,Percent Complete,Percent Pending Investigation,Predicted Value,recreated12m
0,2018-01-01,Alabama,56.0,False,False,AL,2018,January,785,100.0,0.150636,808.0,NaN
48,2018-02-01,Alabama,79.0,False,False,AL,2018,February,799,100.0,0.152024,822.0,NaN
95,2018-03-01,Alabama,76.0,False,False,AL,2018,March,785,100.0,0.154048,808.0,NaN
141,2018-04-01,Alabama,66.0,False,False,AL,2018,April,785,100.0,0.161496,808.0,NaN
187,2018-05-01,Alabama,73.0,False,False,AL,2018,May,787,100.0,0.180146,811.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072,2024-04-01,Wyoming,NaN,NaN,NaN,WY,2024,April,128,100.0,0.019275,132.0,NaN
4079,2024-05-01,Wyoming,NaN,NaN,NaN,WY,2024,May,128,100.0,0.019312,131.0,NaN
4077,2024-06-01,Wyoming,NaN,NaN,NaN,WY,2024,June,127,100.0,0.019212,130.0,NaN
4076,2024-07-01,Wyoming,NaN,NaN,NaN,WY,2024,July,125,100.0,0.019260,127.0,NaN


# Store for viz app

In [15]:
# Store the merged_df DataFrame as a CSV file
merged_df.to_csv('merged_data.csv', index=False)

# Store the merged_df DataFrame as a pickle file
merged_df.to_pickle('merged_df.pkl')

# Confirmation messages
print("Data has been successfully saved to 'merged_data.csv'")
print("Data has been successfully saved to 'merged_df.pkl'")

Data has been successfully saved to 'merged_data.csv'
Data has been successfully saved to 'merged_df.pkl'


<hr>

# Visualizer

In [16]:
selected_state = 'Nevada'

In [17]:
import plotly.graph_objects as go

# Filter merged_df for the selected state's data
graph_data = merged_df[merged_df['State Name'] == selected_state]

# Add hover text with year and month
graph_data = graph_data.copy()  # Prevent SettingWithCopyWarning
graph_data['HoverText'] = graph_data['Date'].dt.strftime('%Y-%B')

# Create the figure
fig = go.Figure()

# Add WONDER Deaths trace
fig.add_trace(go.Scatter(
    x=graph_data['Date'],
    y=graph_data['Deaths'],
    mode='lines',
    name='WONDER Monthly OD Deaths',
    line=dict(color='orange'),
    hovertext=graph_data['HoverText'],
    hoverinfo='text+y',
    yaxis='y1',
    connectgaps=True  # Force display of line even if there is no data
))

# Add Predicted Value trace
fig.add_trace(go.Scatter(
    x=graph_data['Date'],
    y=graph_data['Predicted Value'],
    mode='lines',
    name='12-month Predicted Value',
    line=dict(color='blueviolet'),
    hovertext=graph_data['HoverText'],
    hoverinfo='text+y',
    yaxis='y2',
    connectgaps=True  # Force display of line even if there is no data
))

# Add recreated12m trace
fig.add_trace(go.Scatter(
    x=graph_data['Date'],
    y=graph_data['recreated12m'],
    mode='lines',
    name='Recreated 12-month Cumulative',
    line=dict(color='cornflowerblue'),
    hovertext=graph_data['HoverText'],
    hoverinfo='text+y',
    yaxis='y2',
    connectgaps=True  # Force display of line even if there is no data
))

# Update layout with two y-axes
fig.update_layout(
    title=f'WONDER OD Deaths vs 12-month Predicted Value for {selected_state}',
    yaxis=dict(
        title='WONDER Monthly OD Deaths',
        titlefont=dict(color='orange'),
        tickfont=dict(color='orange'),
        side='left'
    ),
    yaxis2=dict(
        title='12-month Predicted (violet) and Recreated (blue)',
        titlefont=dict(color='cornflowerblue'),
        tickfont=dict(color='blueviolet'),
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5, y=1.1, visible=True, orientation="h", xanchor="center"  # Place the legend above the chart
    ),
    template='plotly_white',
    hovermode='x',
    xaxis=dict(
        title='Date',
        range=['2018-01-01', '2024-12-31']  # Include complete range for 2018 through 2024
    )
)

fig.show()

- Arizona: When do overdoses spike each year? How does the 12-month NVSS cumulative line respond? 

- Montana: Can you identify data drops? The cumulative WONDER count line does not adjust for these, but NVSS does.

- West Virginia: Two large spikes during COVID restriction era drive much of the early trend.

- Colorado: Peak OD may have been in 2021 (or early 2022), but the second peak in 2023 may be a sustained part of the earlier peak.

- Massachusetts: A massive drop starting in 2023 is predicted to continue into 2024.

- Nevada: Are two high OD months in 2023 driving the direction of the predicted trend for 2024?

# Heatmaps

In [18]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle

# Ensure the output directory exists
output_dir = '/work/heatmaps/'
os.makedirs(output_dir, exist_ok=True)

# Generate a heatmap for each state
for state_name in merged_df['State Name'].unique():
    # Filter data for the current state
    state_data = merged_df[merged_df['State Name'] == state_name]
    
    # Prepare pivot table for heatmap
    heatmap_data = state_data.pivot_table(
        index=['State Name'],  # Single row for each state
        columns=state_data['Date'].dt.to_period('M'),  # Columns as year-month
        values='Predicted Value',  # Values to map
        fill_value=0
    )

    # Plot heatmap
    fig, ax = plt.subplots(figsize=(20, 2))
    heatmap = sns.heatmap(heatmap_data, annot=False, fmt=".0f", cmap="Purples", cbar=True, ax=ax)

    # Set background color black and text color white
    plt.gcf().set_facecolor('black')
    ax.tick_params(colors='white')

    # Modify colorbar legend to display white font
    colorbar = heatmap.collections[0].colorbar
    colorbar.ax.yaxis.set_tick_params(color='white')
    plt.setp(colorbar.ax.get_yticklabels(), color='white')

    # Add vertical reference lines at January of each year for better analysis
    heatmap_dates = heatmap_data.columns
    jan_years = [date for date in heatmap_dates if date.start_time.month == 1]

    for date in jan_years:
        x_pos = heatmap_dates.get_loc(date)
        plt.axvline(x=x_pos, color='white', linestyle='-', linewidth=.9)

        # Add year labels at the top of the plot for reference
        year_label = date.start_time.year
        plt.text(x=x_pos + 0.5, y=-0.2, s=str(year_label), color='white', fontsize=15, fontweight='bold', ha='center', va='top', rotation=0)

    # Add an orange empty-fill box from peak month to 11 months earlier
    peak_row = state_data.loc[state_data['Predicted Value'] == state_data['Predicted Value'].max()]
    if not peak_row.empty:
        peak_month_date = peak_row.iloc[0]['Date'].to_period('M')
        if peak_month_date in heatmap_dates:
            peak_month_index = heatmap_dates.get_loc(peak_month_date)
            start_index = max(0, peak_month_index - 11)  # Ensure it doesn't go out of bounds

            rect = Rectangle(
                (start_index, 0),  # Starting X, Y position
                12,  # Width covering 12 months
                heatmap_data.shape[0],  # Height to cover all states (single row here)
                linewidth=2,
                edgecolor='aqua',
                facecolor='none'
            )
            ax.add_patch(rect)

            # Highlight the peak month in orange fill
            rect_fill = Rectangle(
                (peak_month_index, 0),  # Starting X, Y position aligned with peak month
                1,  # Width covering only the peak month
                heatmap_data.shape[0],  # Height to cover all states (single row here)
                linewidth=0,
                edgecolor=None,
                facecolor='aqua',
            )
            ax.add_patch(rect_fill)

    # Adjust X-axis ticks to display abbreviated month names only
    month_labels = [col.strftime('%b')[0] for col in heatmap_dates.to_timestamp()]
    ax.set_xticklabels(month_labels, rotation=0, fontsize=11, color='white')

    # Save as high-resolution PNG
    file_path = os.path.join(output_dir, f"{state_name}.png")
    plt.savefig(file_path, dpi=300, bbox_inches='tight', facecolor='black')
    plt.close(fig)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ce7d8424-642b-490e-b484-ef63058a2a98' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>